In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [7]:
%%sql
WITH
annees AS (
    SELECT CAST(UNNEST(GENERATE_SERIES(2020, 2024)) AS STRING) AS annee
    UNION ALL
    SELECT 'dernier prélévement' AS annee
),

cat AS (
    SELECT DISTINCT categorie
    FROM
        int__mapping_category_simple
    WHERE
        categorie NOT IN (
                'non classé', 'paramètre organoleptique'
            )
)

SELECT DISTINCT
    annees.annee,
    cat.categorie
FROM
    annees
CROSS JOIN
    cat
ORDER BY
    categorie

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie
2024,cvm
2022,cvm
2023,cvm
2021,cvm
2020,cvm
dernier prélévement,cvm
2021,dioxine et furane
2023,dioxine et furane
2020,dioxine et furane
dernier prélévement,dioxine et furane


In [9]:
%%sql
WITH
annees AS (
    SELECT CAST(UNNEST(GENERATE_SERIES(2020, 2024)) AS STRING) AS annee
    UNION ALL
    SELECT 'dernier prélévement' AS annee
),

cat AS (
    SELECT DISTINCT categorie
    FROM
        int__mapping_category_simple
    WHERE
        categorie NOT IN (
                'non classé', 'paramètre organoleptique'
            )
)

SELECT DISTINCT
    annees.annee,
    cat.categorie,
    com.cdreseau
FROM
    int__lien_commune_cdreseau AS com
FULL OUTER JOIN
    annees
ON
    CAST(com.de_partition AS STRING) = annees.annee
CROSS JOIN
    cat
WHERE 
    categorie = 'cvm'
    AND annee = 'dernier prélévement'

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,cdreseau
dernier prélévement,cvm,None


**Il faut des listes de cdreseau différentes pour année et pour last_prélèvement si on veut des lignes coté data qd il n'y a pas de résultats/analyses**

--> Est-ce que le  COALESCE(int__resultats_uid.resultat, 'Pas recherché') AS resultat ne peux pas être fait coté web ?<br>
--> Sinon est-ce qu'il ne faut pas 4 tables au total<br>
    - uid année<br>
    - uid last<br>
    - communes année<br>
    - communes last<br>